In [78]:
from config import views
from spark import createSession

from typing import List

from pyspark.sql.dataframe import DataFrame

import pyspark.sql.functions as F
import pyspark.sql.types as T

from IPython.display import display

In [79]:
def get_columns_of_type(data_frame: DataFrame, type: str) -> List[str]:
    return [column[0] for column in data_frame.dtypes if column[1] == type]

In [80]:
LENGTH = 80
def show_table_name(table: str) -> None:
    print('=' * LENGTH)
    print(' ' * ((LENGTH - len(table)) // 2), table.upper())
    print('=' * LENGTH)

def show_column_name(column: str) -> None:
    print(column.upper())

In [81]:
VERSION = 'v1'

views = views(VERSION)
spark = createSession()

for view, file in views.items():
    df = spark.read.json(file)
    for column in get_columns_of_type(df, 'boolean'):
        df = df.withColumn(column, F.col(column).cast(T.IntegerType()))
    df.createOrReplaceTempView(view)

In [82]:
DATA_FRAMES = list(zip(views.keys(), [spark.sql(f"SELECT * FROM {view}") for view in views.keys()]))

In [83]:
for view, df in DATA_FRAMES:
    show_table_name(view)
    display(df.toPandas())

                                     ARTISTS


,genres,id,name
0,"[filmi, indian folk, indian rock, kannada pop]",72578usTM6Cj5qWsi471Nc,Raghu Dixit
1,"[desi pop, hindi indie, indian indie, indian r...",7b6Ui7JVaBDEfZB9k6nHL0,The Local Train
2,[indian folk],4bvGDTEPFnllKiJaEZGuXk,Achint
3,"[opm, pinoy hip hop, pinoy r&b, pinoy trap, ta...",0n4a5imdLBN24fIrBWoqrv,Because
4,"[hindi indie, indian indie, indian singer-song...",4gdMJYnopf2nEUcanAwstx,Anuv Jain
...,...,...,...
27519,[italian hip hop],2My6j5BEgOi8VHi5WGVyfw,Apocalypshit Army
27520,[belgian pop],0bzW9kGcTyMxXuG9dUdj7E,GRANDGEORGE
27521,[thai indie],4iS19hLpsgRd8jLPKI4Ni3,Blissonic
27522,[thai indie],3JGC3LkYrwlrTscixVwY72,พราว


                                     SESSIONS


,event_type,session_id,timestamp,track_id,user_id
0,PLAY,124,2022-04-19T10:14:08,2FPjk7EjEHD4qgLSSnsWEL,101.0
1,PLAY,124,2022-04-19T10:18:17.973000,6yUmkCTAkHECG4btrYw3cM,101.0
2,SKIP,124,2022-04-19T10:19:03.410000,6yUmkCTAkHECG4btrYw3cM,101.0
3,ADVERTISEMENT,124,2022-04-19T10:19:03.410000,,101.0
4,BUY_PREMIUM,124,2022-04-19T10:19:20.410000,,NaN
...,...,...,...,...,...
3800,PLAY,794,2023-02-19T04:44:21.306000,1A9dNiCCSsiDkIj1RsqMvL,150.0
3801,PLAY,794,2023-02-19T04:47:51.453000,11lkONbH7vsMZEVy012slM,150.0
3802,PLAY,794,2023-02-19T04:51:02.119000,2aMsfiqLC8bMHT6FrrGWY4,150.0
3803,SKIP,794,2023-02-19T04:52:58.495000,2aMsfiqLC8bMHT6FrrGWY4,NaN


                                  TRACK_STORAGE


,daily_cost,storage_class,track_id
0,0.003752,SLOW,708ZiYL3ydBWHS2a7gvJB3
1,0.014561,SLOW,48SFtLr5URCI97X2Ynfdnc
2,0.008304,SLOW,1y0U0HAe5QfTRzOsz74bOt
3,0.012207,SLOW,2TlbZ8JhF9ORa7lJylxABw
4,0.011799,SLOW,7ij5kN8jwXr8fZD54M0xb6
...,...,...,...
129643,0.010739,SLOW,59nszNIEDpnOS0prsKudPb
129644,0.010683,SLOW,0xiHNGGiSfrFfOJZGpxpJY
129645,0.010423,SLOW,4peXvhLT61oP9leXdPQ36B
129646,0.010741,SLOW,2pS2IdtMXpvaEONreUlSAo


                                      TRACKS


,acousticness,danceability,duration_ms,energy,explicit,id,id_artist,instrumentalness,key,liveness,loudness,name,popularity,release_date,speechiness,tempo,valence
0,0.8390,0.740,75040,0.891,0,None,None,0.000000,7,0.8690,-7.480,031 - Der Schatz im Silbersee I - Teil 39,13.0,1968-09-11,0.8920,51.496,0.557
1,0.6950,0.603,291227,0.517,0,48SFtLr5URCI97X2Ynfdnc,2yTUYhIf8fxptTIy3KLuJD,0.000003,6,0.7440,-8.504,Par Avion (Live) ( 2014 - Remaster) - Live; 20...,0.0,2014,0.0235,96.181,0.327
2,0.9530,0.313,166080,0.116,0,1y0U0HAe5QfTRzOsz74bOt,338mC0yGyX0C9of8QMJ5hK,0.331000,0,0.1610,-12.645,My Foolish Heart,25.0,1950-01-01,0.0319,74.071,0.255
3,0.1670,0.958,244133,0.635,0,2TlbZ8JhF9ORa7lJylxABw,5A4ExW2nMBFRy2JDoYUcUE,0.000000,11,0.3620,-7.853,Kathysterisi,14.0,1998,0.2590,108.024,0.866
4,0.1200,0.684,235974,0.839,0,7ij5kN8jwXr8fZD54M0xb6,48CUA59SDed3IdCctKndud,0.000000,4,0.3540,-6.457,Aleni Aleni,51.0,2015,0.0658,128.051,0.580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129643,0.4110,0.633,214773,0.345,0,59nszNIEDpnOS0prsKudPb,6wcIBaOvA9XNGgPujYZZ7L,0.000028,4,0.3610,-15.231,最真的夢,16.0,1990-02-05,0.0291,132.691,0.368
129644,0.2220,0.295,213667,0.417,0,0xiHNGGiSfrFfOJZGpxpJY,04u3fc37nHFKN7GJTSIwI8,0.000006,6,0.1480,-8.002,By My Side,61.0,2017-08-11,0.0307,64.687,0.135
129645,0.6720,0.347,208467,0.216,0,4peXvhLT61oP9leXdPQ36B,4etuCZVdP8yiNPn4xf0ie5,0.000118,8,0.0738,-15.215,Cu Cu Rru Cu Cu Paloma,49.0,1978,0.0315,108.566,0.478
129646,0.0229,0.784,214827,0.821,0,2pS2IdtMXpvaEONreUlSAo,6IE6z7DcZIT4Ml3Fh5Ivch,0.000007,0,0.1760,-7.621,No Quiero Saber - 2000 Mix,26.0,1990,0.0423,119.609,0.885


                                      USERS


,city,favourite_genres,id,name,premium_user,street,user_id
0,Warszawa,"[motown, soul, regional mexican]",NaN,Marika Pilipczuk,1.0,ul. Księżycowa 31,101
1,Gdynia,"[regional mexican, psychedelic rock, new roman...",NaN,Anita Pioch,1.0,plac Sadowa 527,102
2,Kraków,"[soul, mellow gold, blues rock]",NaN,Jan Gryga,1.0,plac Wyspiańskiego 73/43,103
3,Wrocław,"[permanent wave, post-teen pop, mandopop]",NaN,Ksawery Klus,1.0,ulica Długosza 71/06,104
4,Gdynia,"[metal, new wave, argentine rock]",NaN,Maciej Bandyk,1.0,ul. Rybacka 07,105
5,Kraków,None,NaN,Nikodem Kopciuch,1.0,pl. Promienna 59/43,106
6,Poznań,"[europop, folk, tropical]",NaN,Kacper Osojca,1.0,pl. Staszica 343,107
7,Warszawa,"[new wave, psychedelic rock, soft rock]",NaN,Maurycy Szoka,1.0,al. Tęczowa 332,108
8,Szczecin,"[roots rock, latin pop, alternative metal]",NaN,Sebastian Molka,1.0,al. Armii Krajowej 564,109
9,Kraków,"[lounge, hoerspiel, album rock]",NaN,Filip Kalinka,1.0,aleja Bema 889,110


In [84]:
for view, data_frame in DATA_FRAMES:
    show_table_name(view)
    for column, type in data_frame.dtypes:
        show_column_name(column)
        group_by_column = f"""--sql
            SELECT 
                {column},
                COUNT(*) AS length
            FROM {view}
            GROUP BY {column}
            ORDER BY {column} IS NULL DESC, length DESC, {column} NULLS FIRST
        """
        df = spark.sql(group_by_column)
        display(df.toPandas())

        count_distinct = f"""--sql
            SELECT
                COUNT(DISTINCT {column})
            FROM {view}
        """
        df = spark.sql(count_distinct)
        display(df.toPandas())

                                     ARTISTS
GENRES


,genres,length
0,None,1352
1,[indonesian pop],74
2,[classic thai pop],68
3,[thai pop],60
4,[classic turkish pop],57
...,...,...
13066,[yiddish folk],1
13067,[yoga],1
13068,[yugoslav new wave],1
13069,[zhongguo feng],1


,count(DISTINCT genres)
0,13070


ID


,id,length
0,-1,1371
1,0001wHqxbF2YYRQxGdbyER,1
2,000p4jMMhpEHq1h6PFCyO1,1
3,001aJOc7CSQVo3XzoLG4DK,1
4,0027wHZDQXpRll4ckwDGad,1
...,...,...
26149,7zup4xIPjtv50lM7x3n4qW,1
26150,7zw8gWmNncuk2QZHIc70So,1
26151,7zwF847GE2hY5ApGSOLmBG,1
26152,7zwiFdY90oXzLh1Wz22oEq,1


...

,street,length
0,al. Armii Krajowej 564,1
1,al. Jesionowa 47,1
2,al. Konwaliowa 33,1
3,al. Okrzei 69,1
4,al. Podleśna 00,1
5,al. Szeroka 27/38,1
6,al. Tęczowa 332,1
7,aleja Bema 889,1
8,aleja Prusa 830,1
9,aleja Stolarska 554,1


,count(DISTINCT street)
0,50


USER_ID


,user_id,length
0,101,1
1,102,1
2,103,1
3,104,1
4,105,1
5,106,1
6,107,1
7,108,1
8,109,1
9,110,1


,count(DISTINCT user_id)
0,50


In [88]:
def aggregate_numeric_column(view: str, column: str) -> str:
    return f"""--sql
            SELECT
                "{column}" AS name,
                COUNT({column}) AS count,
                MIN({column}) AS min,
                MAX({column}) AS max,
                AVG({column}) AS average,
                SUM({column}) AS sum,
                SUM(DISTINCT {column}) AS sumDistinct,
                KURTOSIS({column}) AS kurtosis,
                SKEWNESS({column}) AS skewness,
                STDDEV({column}) AS standard_deviation,
                STDDEV_POP({column}) AS population_standard_deviation,
                VARIANCE({column}) AS variance,
                VAR_POP({column}) AS population_variance
            FROM {view}
            WHERE {column} IS NOT NULL
        """

for view, data_frame in DATA_FRAMES:
    show_table_name(view)
    for column, type in data_frame.dtypes:
        if type in ['double', 'bigint']:
            show_column_name(column)
            df = spark.sql(aggregate_numeric_column(view, column))
            display(df.toPandas())

                                     ARTISTS
                                     SESSIONS
SESSION_ID


,column,count,min,max,average,sum,sumDistinct,kurtosis,skewness,standard_deviation,population_standard_deviation,variance,population_variance
0,session_id,3805,124,794,461.138765,1754633,285854,-1.280354,0.018784,197.815899,197.789903,39131.130056,39120.845922


USER_ID


,column,count,min,max,average,sum,sumDistinct,kurtosis,skewness,standard_deviation,population_standard_deviation,variance,population_variance
0,user_id,3622,101,150,126.139978,456879,6275,-1.331799,-0.042814,14.900369,14.898312,222.020997,221.959699


                                  TRACK_STORAGE
DAILY_COST


...

,column,count,min,max,average,sum,sumDistinct,kurtosis,skewness,standard_deviation,population_standard_deviation,variance,population_variance
0,valence,129648,0.0,1.0,0.563443,73049.2694,537.05768,-1.035815,-0.154964,0.252581,0.25258,0.063797,0.063796


                                      USERS
ID


,column,count,min,max,average,sum,sumDistinct,kurtosis,skewness,standard_deviation,population_standard_deviation,variance,population_variance
0,id,5,-1,-1,-1.0,-5,-1,NaN,NaN,0.0,0.0,0.0,0.0


USER_ID


,column,count,min,max,average,sum,sumDistinct,kurtosis,skewness,standard_deviation,population_standard_deviation,variance,population_variance
0,user_id,50,101,150,125.5,6275,6275,-1.20096,2.542155e-16,14.57738,14.43087,212.5,208.25


In [92]:
def explode_column(view: str, column: str) -> str:
    return f"""--sql
            SELECT
                DISTINCT EXPLODE({column}) AS distinct_{column}
            FROM {view}
            ORDER BY distinct_{column} NULLS FIRST
        """


def count_exploded_column(view: str, column: str) -> str:
    exploded = f"""--sql
        SELECT
            DISTINCT EXPLODE({column}) AS {column}
        FROM {view}
    """

    return f"""--sql
            SELECT
                COUNT(*) AS length
            FROM ({exploded})
        """

for view, data_frame in DATA_FRAMES:
    show_table_name(view)
    for column, type in data_frame.dtypes:
        if type.startswith('array'):
            show_column_name(column)
            df = spark.sql(explode_column(view, column))
            display(df.toPandas())
            df = spark.sql(count_exploded_column(view, column))
            display(df.toPandas())

                                     ARTISTS
GENRES


,distinct_genres
0,48g
1,a cappella
2,abstract
3,abstract hip hop
4,accordeon
...,...
3867,zolo
3868,zouglou
3869,zouk
3870,zouk riddim


,length
0,3872


                                     SESSIONS
                                  TRACK_STORAGE
                                      TRACKS
                                      USERS
FAVOURITE_GENRES


,distinct_favourite_genres
0,adult standards
1,album rock
2,alternative metal
3,alternative rock
4,argentine rock
5,art rock
6,blues rock
7,brill building pop
8,c-pop
9,classic rock


,length
0,46
